In [2]:
from qwikidata.json_dump import WikidataJsonDump
from pprint import pprint
from tqdm import tqdm
import pandas as pd

wjd = WikidataJsonDump('/workdir/important_datasets/wikidata-20220103-all.json.gz')

In [3]:
from qwikidata.sparql import return_sparql_query_results

query_string = """
SELECT ?WDid
WHERE {
  ?WDid wdt:P105 wd:Q7432.  # Taxon rank is species
  ?WDid wdt:P171 wd:Q756.   # Kingdom is plants
}
"""

results = return_sparql_query_results(query_string)
print(len(results['results']['bindings']))

0


In [6]:
query_string = """
SELECT ?WDid ?WDidLabel ?property ?propertyLabel
WHERE {
  ?WDid ?property wd:Q15254139.  # Any property pointing to Q15254139
  SERVICE wikibase:label {       # Fetch labels for entities and properties
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
  }
}
LIMIT 100
"""

results = return_sparql_query_results(query_string)
for result in results['results']['bindings']:
    print(f"Entity: {result['WDidLabel']['value']} (ID: {result['WDid']['value']})")
    print(f"Property: {result['propertyLabel']['value']} (ID: {result['property']['value']})")
    print("---")


Entity: statement/Q497975-23480ba8-23b7-4507-ae17-b7b3d8d00948 (ID: http://www.wikidata.org/entity/statement/Q497975-23480ba8-23b7-4507-ae17-b7b3d8d00948)
Property: http://www.wikidata.org/prop/statement/P910 (ID: http://www.wikidata.org/prop/statement/P910)
---
Entity: https://ar.wikipedia.org/wiki/%D8%AA%D8%B5%D9%86%D9%8A%D9%81:%D8%AB%D9%85%D8%A7%D8%B1_%D8%B5%D8%A7%D9%84%D8%AD%D8%A9_%D9%84%D9%84%D8%A3%D9%83%D9%84 (ID: https://ar.wikipedia.org/wiki/%D8%AA%D8%B5%D9%86%D9%8A%D9%81:%D8%AB%D9%85%D8%A7%D8%B1_%D8%B5%D8%A7%D9%84%D8%AD%D8%A9_%D9%84%D9%84%D8%A3%D9%83%D9%84)
Property: http://schema.org/about (ID: http://schema.org/about)
---
Entity: https://az.wikipedia.org/wiki/Kateqoriya:Yem%C9%99li_meyv%C9%99l%C9%99r (ID: https://az.wikipedia.org/wiki/Kateqoriya:Yem%C9%99li_meyv%C9%99l%C9%99r)
Property: http://schema.org/about (ID: http://schema.org/about)
---
Entity: https://be-tarask.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D1%8D%D0%B3%D0%BE%D1%80%D1%8B%D1%8F:%D0%AF%D0%B4%D0%BE%D0%BC%D1%8B%D1

In [12]:
from qwikidata.sparql import return_sparql_query_results

query_string = '''
SELECT ?WDid ?EPPOCode
WHERE {
  ?WDid wdt:P3031 ?EPPOCode.  # Has a EPPO ID
}
'''

results = return_sparql_query_results(query_string)
print(len(results['results']['bindings']))

69770


In [13]:
results['results']['bindings']

EPPOCodes = []

for result in results['results']['bindings']:
    EPPOCodes.append(result['EPPOCode']['value'])

In [16]:
code1 = []

for code in EPPOCodes:
    if code.startswith('1'):
        code1.append(code)
        
print(len(code1)) 

17911


In [29]:
EPPOCodes = list(set(EPPOCodes))

In [19]:
df_EPPO = pd.read_parquet('/workdir/EPPO Dataframes/df.plants.scraped.sorted.gzip',engine='fastparquet')

In [25]:
db_EPPOCodes = list(df_EPPO['EPPO Code'])

In [32]:
check = 0

for i, code in enumerate(db_EPPOCodes):
    if code not in EPPOCodes:
        print(i, ':' , code)
        check += 1
    if check > 500:
        break

16 : CIDSS
23 : CIDLO
25 : CIDNO
34 : CIDHX
43 : CIDMY
46 : CIDSW
58 : CIDSP
60 : CIDPE
62 : CIDAY
64 : CIDLC
66 : CIDRX
73 : CIDHL
82 : ROSSS
113 : PIUSS
140 : PIUTA
144 : PRNSS
179 : PIUDO
188 : PIUTC
204 : PIUPM
208 : PIUCU
209 : PIUDN
213 : PIUKK
218 : PIURU
223 : PIUSO
224 : PIUBM
228 : PIUBP
230 : PIUHR
232 : PIUEE
238 : PIUTM
247 : PIUKL
256 : MUBSS
267 : COFSS
270 : GOSSS
292 : COFST
293 : COFEX
308 : COFHY
309 : COFKA
311 : COFEU
323 : MUBTR
344 : ROSHY
352 : ROSID
362 : ROSOP
380 : RUBSS
386 : PRNIA
391 : PRNOT
393 : PRNAS
400 : PRNAY
403 : PRNHY
410 : ROSXX
413 : PRNFM
418 : PRNLS
422 : PRNCG
423 : PRNPH
428 : PRNPQ
436 : PRNHL
439 : PRNCU
442 : PRNZB
450 : QUESS
455 : GOSTU
460 : PYUSS
467 : MABSS
468 : SAXSS
474 : BRSSS
479 : CUUSS
505 : SOLMQ
518 : SOLTH
519 : BRSOH
533 : PIEJJ
534 : SOLVL
536 : EUCSS
539 : SOLOX
557 : SOLVC
560 : CUUMZ
561 : SOLRP
568 : PIEJH
571 : PIEKR
578 : SOLTL
584 : POPSS
589 : SOLXN
593 : SOLSS
612 : CUUPG
615 : SOLVR
619 : SOLAI
621 : CUUCY
623 :

In [ ]:
query_string = '''
SELECT ?WDid ?GBIFCode
WHERE {
  ?WDid wdt:P105 wd:Q7432.  # Taxon rank is species
  ?WDid wdt:P3031 ?EPPOCode.  # Has a EPPO taxon ID
}
'''

results = return_sparql_query_results(query_string)
print(len(results['results']['bindings']))



In [38]:
len(results['results']['bindings'])

3312808

In [7]:
for i, entity_dict in tqdm(enumerate(wjd)):
    # print(list(entity_dict.keys()))
    # print(list(entity_dict['claims'].keys()))
    # pprint(entity_dict)
    if entity_dict['id'].startswith('P'):
        found = entity_dict
        break    
    if i > 1000000:
        print('Not found')
        break

157701it [00:43, 3634.10it/s]


KeyboardInterrupt: 

In [6]:
from qwikidata.entity import WikidataItem, WikidataProperty

type_to_entity_class = {"item": WikidataItem, "property": WikidataProperty}
max_entities = 2
entities = []


for ii, entity_dict in enumerate(wjd):
  if ii >= max_entities:
    break
  entity_id = entity_dict["id"]
  entity_type = entity_dict["type"]
  entity = type_to_entity_class[entity_type](entity_dict)
  entities.append(entity)

for entity in entities:
  print(entity)


WikidataItem(label=Belgium, id=Q31, description=country in western Europe, aliases=['Kingdom of Belgium', 'BEL', 'be', '🇧🇪', 'BE'], enwiki_title=Belgium)
WikidataItem(label=happiness, id=Q8, description=mental or emotional state of well-being characterized by pleasant emotions, aliases=['joy', 'happy', 'Happiness'], enwiki_title=Happiness)
